### Preparación de datos

In [2]:
import pandas as pd
from pathlib import Path
import re
# === RUTAS ===

ruta_acta =  Path(r"20251003/20251003/ADP_DTM_FACT.Acta.csv")
ruta_insumo =  Path(r"20251003/20251003/ADP_DTM_DIM.Insumo.csv")
ruta_proyectos =  Path(r"20251003/20251003/ADP_DTM_DIM.Proyecto.csv")
ruta_capitulos = Path(r"20251003/20251003/ADP_DTM_DIM.CapituloPresupuesto.csv")
ruta_proyeccion =  Path(r"20251003/20251003/ADP_DTM_FACT.Proyeccion.csv")
ruta_items =  Path(r"20251003/20251003/ADP_DTM_DIM.Items.csv")
ruta_macro = Path(r"20251003/20251003/proyectos_macroproyectos.csv")
tabla_macro = pd.read_csv(ruta_macro)


In [3]:
# === CARGA ===
tabla_proyeccion = pd.read_csv(ruta_proyeccion)
tabla_items = pd.read_csv(ruta_items)
tabla_proyectos = pd.read_csv(ruta_proyectos)
tabla_capitulos = pd.read_csv(ruta_capitulos)
tabla_insumos = pd.read_csv(ruta_insumo)
tabla_macro = pd.read_csv(ruta_macro)

C:\Users\aleja\AppData\Local\Temp\ipykernel_24568\1549343039.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  tabla_items = pd.read_csv(ruta_items)


#### New Join: (macroproyectos incluidos)

In [4]:
# Alinear columnas
if "Proyectos" in tabla_macro.columns:
    tabla_macro = tabla_macro.rename(columns={"Proyectos": "Nombre Proyecto"})
print("[DEBUG] Columnas alineadas.")


# Crear columna limpia sin prefijo en tabla_macro
tabla_macro["Proyecto_sin_prefijo"] = tabla_macro["Proyecto"].apply(
    lambda x: re.sub(r"^\s*\d+\s*-\s*", "", str(x)).strip() if pd.notna(x) else x
)

# Verificar coincidencias
interseccion = set(tabla_proyectos["Nombre Proyecto"]).intersection(set(tabla_macro["Proyecto_sin_prefijo"]))
print(f"[DEBUG] Coincidencias exactas entre tablas: {len(interseccion)}")

# === Merge de proyectos con macroproyectos ===
tabla_proyectos_macro = pd.merge(
    tabla_proyectos,
    tabla_macro,
    left_on="Nombre Proyecto",
    right_on="Proyecto_sin_prefijo",
    how="left",
)

print(f"[DEBUG] Merge proyectos + macroproyectos: {tabla_proyectos_macro.shape}")

# === PIPELINE COMPLETO ===
tabla_base = tabla_proyeccion.copy()

tabla_1 = pd.merge(tabla_base, tabla_proyectos_macro, on="SkIdProyecto", how="left")
tabla_2 = pd.merge(tabla_1, tabla_capitulos, on="SkIdCapitulo", how="left")

# Items
tabla_items_unica = tabla_items.drop_duplicates(subset=["SkIdItems"], keep="first")
tabla_3 = pd.merge(tabla_2, tabla_items_unica, on="SkIdItems", how="left", suffixes=("", "_item"))

# Insumos
tabla_insumos_unica = tabla_insumos.drop_duplicates(subset=["SkIdInsumo"], keep="first").copy()
cols_no_clave = [c for c in tabla_insumos_unica.columns if c != "SkIdInsumo"]
tabla_insumos_pref = tabla_insumos_unica.rename(columns={c: f"Insumo_{c}" for c in cols_no_clave})

tabla_join = pd.merge(tabla_3, tabla_insumos_pref, on="SkIdInsumo", how="left")
print(f"[DEBUG] OK. Forma final: {tabla_join.shape}")


'''
# === Exportar resultados ===
tabla_join.to_csv("tabla_looker.csv", index=False, encoding="utf-8", sep=",")
tabla_join.to_parquet("tabla_looker.parquet", index=False)
print("[DEBUG] Exportado CSV y Parquet correctamente.")
'''


[DEBUG] Columnas alineadas.
[DEBUG] Coincidencias exactas entre tablas: 65
[DEBUG] Merge proyectos + macroproyectos: (85, 44)
[DEBUG] OK. Forma final: (273450, 114)


'\n# === Exportar resultados ===\ntabla_join.to_csv("tabla_looker.csv", index=False, encoding="utf-8", sep=",")\ntabla_join.to_parquet("tabla_looker.parquet", index=False)\nprint("[DEBUG] Exportado CSV y Parquet correctamente.")\n'

In [5]:
## Vista de Concatenación Jerárquica
# Mostrar solo algunas columnas relevantes
columnas_mostrar = ["Macroproyecto", "Proyecto_sin_prefijo","Item Descripcion","Insumo_Insumo Descripcion"]
vista_concatenacion = tabla_join[columnas_mostrar]


### Verificación

In [5]:
# Vista rápida de verificación

# Obtener conjuntos únicos
set_macroproyectos = set(vista_concatenacion["Proyecto_sin_prefijo"].dropna().str.strip())
# Filtrar filas donde el campo 'Macroproyecto' está vacío o no coincide con los esperados
filtro_macro = vista_concatenacion[vista_concatenacion["Macroproyecto"].notna() | (vista_concatenacion["Macroproyecto"].str.strip() == "")]


display(filtro_macro[columnas_mostrar].head(10))

,Macroproyecto,Proyecto_sin_prefijo,Item Descripcion,Insumo_Insumo Descripcion
10273,Caminos de Sie - Manzana 4,Caminos de Sie - Urb y Zonas Comunes MZ 4,Relleno en recebo base tanque,recebo B-200
10274,Caminos de Sie - Manzana 4,Caminos de Sie - Urb y Zonas Comunes MZ 4,Relleno en recebo base tanque,mano de obra preliminares ug
10275,Caminos de Sie - Manzana 4,Caminos de Sie - Urb y Zonas Comunes MZ 4,Relleno en recebo base tanque,alquiler vibrocompactador 3T
10276,Caminos de Sie - Manzana 4,Caminos de Sie - Urb y Zonas Comunes MZ 4,Relleno en recebo contra muros tanque,recebo B-200
10277,Caminos de Sie - Manzana 4,Caminos de Sie - Urb y Zonas Comunes MZ 4,Relleno en recebo contra muros tanque,mano de obra cimentación ug
12252,Caminos de Sie - Manzana 4,Caminos de Sie - Urb y Zonas Comunes MZ 4,Imprevistos,imprevisto de obra
12358,Caminos de Sie - Manzana 4,Caminos de SIE - Edificaciones MZ 4,Excavación a mano cimentación con retiro,mano de obra cimentación ug
12359,Caminos de Sie - Manzana 4,Caminos de SIE - Edificaciones MZ 4,Relleno en recebo a mano cimentación,mano de obra cimentación ug
12360,Caminos de Sie - Manzana 4,Caminos de SIE - Edificaciones MZ 4,Placa contrapiso e=0.15m,mano de obra cimentación ug
12361,Caminos de Sie - Manzana 4,Caminos de SIE - Edificaciones MZ 4,Anden perimetral,mano de obra cimentación ug


In [6]:
tabla_join.shape


(273450, 114)

In [6]:
# === SELECCIÓN DE COLUMNAS ===
columnas_finales = [
    "SkIdProyecto", "SkIdCapitulo", "SkIdItems", "SkIdInsumo",
    "Nombre Proyecto", "Capitulo Descripcion", "Item Descripcion",
    "Insumo_Insumo Descripcion", "Insumo_Agrupacion Descripcion",
    "SkIdFecha Real", "Cantidad", "Valor Unitario", "Valor Total",
    "Insumo_Valor Unitario", "Insumo_Valor Neto", "Insumo_Fecha Creacion",
    "Cantidad Item", "Macroproyecto", "Insumo_Fecha Modificacion",
    "Fecha De Elaboracion", "Fecha De Inicio", "Fecha De Finalización",
    "SkIdFecha", "Capitulo Numero", "Cantidad_Item"
]

# Filtrar solo las columnas que existan realmente (por seguridad)
columnas_existentes = [col for col in columnas_finales if col in tabla_join.columns]
tabla_looker = tabla_join[columnas_existentes].copy()

print(f"[DEBUG] Columnas seleccionadas: {len(columnas_existentes)} / {len(columnas_finales)}")
print(f"[DEBUG] Forma final de tabla_looker: {tabla_looker.shape}")
display(tabla_looker.head(100))


[DEBUG] Columnas seleccionadas: 23 / 25
[DEBUG] Forma final de tabla_looker: (273450, 23)


,SkIdProyecto,SkIdCapitulo,SkIdItems,SkIdInsumo,Nombre Proyecto,Capitulo Descripcion,Item Descripcion,Insumo_Insumo Descripcion,Insumo_Agrupacion Descripcion,SkIdFecha Real,...,Insumo_Valor Unitario,Insumo_Valor Neto,Insumo_Fecha Creacion,Cantidad Item,Macroproyecto,Insumo_Fecha Modificacion,Fecha De Elaboracion,Fecha De Inicio,SkIdFecha,Capitulo Numero
0,1005,1005100,1006583,100279,URBAN PLAZA,CIMENTACION,Pilote preexcavado d=80cm,agua carrotanque,Campamento y provisionales,20110929,...,7327.59,8719.8321,06/12/2010,2.00,NaN,13/01/2011,28/03/2011,25/01/2011,20110929,2
1,1005,1005100,1006583,1002087,URBAN PLAZA,CIMENTACION,Pilote preexcavado d=80cm,concreto tremie 3000 psi grava común,Concretos Especiales,20110929,...,263115.84,313107.8496,06/12/2010,2.00,NaN,NaN,28/03/2011,25/01/2011,20110929,2
2,1005,1005100,1006583,1006063,URBAN PLAZA,CIMENTACION,Pilote preexcavado d=80cm,subcontrato pilote preexcavado d=0.80m,Subcontratos Preliminares Cimentación y Estruc...,20110929,...,40250.00,40250.0000,06/12/2010,2.00,NaN,27/05/2011,28/03/2011,25/01/2011,20110929,2
3,1005,1005100,1007643,100279,URBAN PLAZA,CIMENTACION,Pantalla preexcavada,agua carrotanque,Campamento y provisionales,20110930,...,7327.59,8719.8321,06/12/2010,32.28,NaN,13/01/2011,28/03/2011,25/01/2011,20110930,2
4,1005,1005100,1007643,1002087,URBAN PLAZA,CIMENTACION,Pantalla preexcavada,concreto tremie 3000 psi grava común,Concretos Especiales,20110930,...,263115.84,313107.8496,06/12/2010,32.28,NaN,NaN,28/03/2011,25/01/2011,20110930,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1005,1005102,1005464,1002329,URBAN PLAZA,ESTRUCTURA,Escalera en concreto - Zona subterranea,desmoldante (p),Desmoldantes y Curadores,20111021,...,7727.59,9195.8321,06/12/2010,22.34,NaN,NaN,28/03/2011,25/01/2011,20111021,4
96,1005,1005102,1005464,1002400,URBAN PLAZA,ESTRUCTURA,Escalera en concreto - Zona subterranea,distanciador silla mortero 25 mm,Accesorios Técnicos para Concreto,20111021,...,260.00,309.4000,06/12/2010,22.34,NaN,NaN,28/03/2011,25/01/2011,20111021,4
97,1005,1005102,1005464,1003845,URBAN PLAZA,ESTRUCTURA,Escalera en concreto - Zona subterranea,mano de obra estructura,Mano de Obra Estructura,20111021,...,2978.00,2978.0000,06/12/2010,22.34,NaN,26/05/2011,28/03/2011,25/01/2011,20111021,4
98,1005,1005102,1005464,1005022,URBAN PLAZA,ESTRUCTURA,Escalera en concreto - Zona subterranea,"puntilla con cabeza de 1-1/2"" a 4"" (p)",Demás Elementos de Ferretería,20111021,...,1350.00,1606.5000,06/12/2010,22.34,NaN,29/12/2022,28/03/2011,25/01/2011,20111021,4


#### Debug

In [8]:
# === MOSTRAR NOMBRES DE PROYECTOS Y MACROPROYECTOS ===

# Obtener listas únicas ordenadas
nombres_proyectos = sorted(tabla_proyectos["Nombre Proyecto"].dropna().unique().tolist())
nombres_macro = sorted(tabla_macro["Proyecto"].dropna().unique().tolist())

# Convertir a DataFrame comparativo
longitud_max = max(len(nombres_proyectos), len(nombres_macro))
tabla_comparativa = pd.DataFrame({
    "Nombre Proyecto (tabla_proyectos)": nombres_proyectos + [""] * (longitud_max - len(nombres_proyectos)),
    "Proyecto (proyectos_macroproyectos.csv)": nombres_macro + [""] * (longitud_max - len(nombres_macro))
})

print(f"[DEBUG] Proyectos en tabla_proyectos: {len(nombres_proyectos)}")
print(f"[DEBUG] Proyectos en proyectos_macroproyectos.csv: {len(nombres_macro)}")
print(f"[DEBUG] Mostrando tabla comparativa de nombres (orden alfabético)")

display(tabla_comparativa)


[DEBUG] Proyectos en tabla_proyectos: 85
[DEBUG] Proyectos en proyectos_macroproyectos.csv: 74
[DEBUG] Mostrando tabla comparativa de nombres (orden alfabético)


,Nombre Proyecto (tabla_proyectos),Proyecto (proyectos_macroproyectos.csv)
0,El Polo 1 - Etapa 1 - Torre 1,102 - Caminos de Sie - Urbanismo Externo MZ 2
1,ATRIO - Torre Norte,104 - Caminos de SIE - Edificaciones MZ2 - Et 1
2,BS Rosales,105 - Caminos de SIE - Edificaciones MZ2 - Et 2
3,CC Atrio - Circundantes Plaza Cívica,106 - Caminos de SIE - Edificaciones MZ2 - Et 3
4,CC Atrio - Plaza Cívica,107 - Caminos de SIE - Edificaciones MZ2 - Et 4
...,...,...
80,Valverde - Etapa Menta,
81,Valverde - Etapa Olivo,
82,Valverde - Palma,
83,Valverde - Roble,


#### quitar prefijo para ajustar nombres de macroproyectos

In [9]:
# === COMPARAR NOMBRES LIMPIOS DE MACROPROYECTOS VS tabla_proyectos ===
import pandas as pd
import re

def quitar_prefijo_numero(nombre: str) -> str:
    if pd.isna(nombre):
        return ""
    return re.sub(r"^\s*\d+\s*-\s*", "", str(nombre)).strip()

# Crear columna limpia en la tabla de macroproyectos
tabla_macro["Proyecto_sin_prefijo"] = tabla_macro["Proyecto"].apply(quitar_prefijo_numero)

# Obtener conjuntos únicos
set_proyectos = set(tabla_proyectos["Nombre Proyecto"].dropna().str.strip())
set_macro_limpio = set(tabla_macro["Proyecto_sin_prefijo"].dropna().str.strip())

# Calcular intersección y diferencias
interseccion = set_proyectos.intersection(set_macro_limpio)
faltantes_en_macro = set_proyectos - set_macro_limpio
faltantes_en_proyectos = set_macro_limpio - set_proyectos

print(f"[DEBUG] Coincidencias exactas (sin prefijo): {len(interseccion)}")
print(f"[DEBUG] Faltan en macroproyectos.csv (aun después de limpiar): {len(faltantes_en_macro)}")
print(f"[DEBUG] Faltan en tabla_proyectos: {len(faltantes_en_proyectos)}")

# Mostrar ejemplos de ambos
print("\n[DEBUG] Ejemplos de coincidencias:")
print(sorted(list(interseccion))[:5])

print("\n[DEBUG] Ejemplos de proyectos que faltan en macroproyectos:")
print(sorted(list(faltantes_en_macro))[:5])

print("\n[DEBUG] Ejemplos de proyectos que faltan en tabla_proyectos:")
print(sorted(list(faltantes_en_proyectos))[:5])


[DEBUG] Coincidencias exactas (sin prefijo): 67
[DEBUG] Faltan en macroproyectos.csv (aun después de limpiar): 18
[DEBUG] Faltan en tabla_proyectos: 7

[DEBUG] Ejemplos de coincidencias:
['BS Rosales', 'Caminos de SIE - Edificaciones MZ 4', 'Caminos de Sie - Edificaciones MZ1 - Et 1', 'Caminos de Sie - Edificaciones MZ1 - Et 2', 'Caminos de Sie - Edificaciones MZ1 - Et 3']

[DEBUG] Ejemplos de proyectos que faltan en macroproyectos:
['ATRIO - Torre Norte', 'CC Atrio - Circundantes Plaza Cívica', 'CC Atrio - Plaza Cívica', 'Caminos de Sie - Urbanismo Externo', 'Centro Cultural Atrio - Etapa 1']

[DEBUG] Ejemplos de proyectos que faltan en tabla_proyectos:
['Caminos de SIE - Edificaciones MZ2 - Et 1', 'Caminos de SIE - Edificaciones MZ2 - Et 2', 'Caminos de SIE - Edificaciones MZ2 - Et 3', 'Caminos de SIE - Edificaciones MZ2 - Et 4', 'Caminos de Sie - Urbanismo Externo MZ 2']


In [10]:
import re

if "Nombre Proyecto" in tabla_looker.columns and "Proyecto_sin_prefijo" not in tabla_looker.columns:
    tabla_looker["Proyecto_sin_prefijo"] = tabla_looker["Nombre Proyecto"].apply(
        lambda x: re.sub(r"^\s*\d+\s*-\s*", "", str(x)).strip() if pd.notna(x) else x
    )
    print("[DEBUG] Columna 'Proyecto_sin_prefijo' creada en tabla_looker.")

[DEBUG] Columna 'Proyecto_sin_prefijo' creada en tabla_looker.


## Juego

### Comienza

In [ ]:
set(tabla_join["Capitulo Numero"])

In [ ]:
set(tabla_join["Capitulo Descripcion"])

In [ ]:
tabla_join["Capitulo Numero"].value_counts()

In [ ]:
 tabla_join[["Capitulo Numero", "Capitulo Descripcion"]].drop_duplicates().sort_values("Capitulo Numero").reset_index(drop=True).value_counts()

In [7]:
# --- Convert pairs to list of "Numero,Descripcion" strings ---
pares_lista = (
    tabla_join[["Capitulo Numero", "Capitulo Descripcion"]]
    .drop_duplicates()
    .apply(lambda fila: f"{fila['Capitulo Numero']}. {fila['Capitulo Descripcion']}", axis=1)
    .tolist()
)

print(pares_lista)


['2. CIMENTACION', '24. EQUIPOS Y HERRAMIENTAS', '38. INDIRECTOS', '4. ESTRUCTURA', '26. ADMINISTRACION DE OBRA', '1. PRELIMINARES', '5. MAMPOSTERIA', '3. DESAGÜES', '6. PAÑETES', '9. PISOS', '16. CARPINTERIA METALICA', '29. ALCANTARILLADOS', '30. ACUEDUCTOS', '31. VIAS Y ANDENES', '34. ZONAS VERDES, CERRAMIENTO Y ESPACIOS COMUNALES', '7. CUBIERTAS Y CIELORASOS', '23. REMATES Y ASEO', '10. ENCHAPES', '25. GASTOS GENERALES', '11. INSTALACIONES ELECTRICAS Y DE COMUNICACION', '12. INSTALACIONES HIDROSANITARIAS', '8. IMPERMEABILIZACIONES', '18. EQUIPOS ESPECIALES', '17. APARATOS SANITARIOS Y DE COCINA', '15. CARPINTERIA MADERA', '20. CERRADURAS, ESPEJOS Y NOMENCLATURA', '19. PINTURA', '13. INSTALACIONES DE GAS', '21. EXTERIORES', '37. IMPREVISTOS', '14. INSTALACIONES MECANICAS', '32. REDES ELECTRICAS Y DE COMUNICACION', '22. EQUIPAMIENTO COMUNAL', '36. POSTVENTAS', '35. MOBILIARIO', '21. PAISAJISMO', '7. CIELORASOS', '12. VENTANERIAS', '14. CARPINTERIA MADERA', '20. INSTALACIONES ELECTRICA

### carga de embeddings

In [33]:
homologables = pd.read_parquet("embeddings.parquet")


In [45]:
homologables.head()

,text,vector
0,2. CIMENTACION,"[0.028495730832219124, 0.03362597897648811, 0...."
1,24. EQUIPOS Y HERRAMIENTAS,"[0.030140655115246773, 0.05396698787808418, -0..."
2,38. INDIRECTOS,"[0.016599023714661598, 0.04692142829298973, -0..."
3,4. ESTRUCTURA,"[0.012884092517197132, 0.02800784818828106, 0...."
4,26. ADMINISTRACION DE OBRA,"[0.038438744843006134, 0.049190178513526917, 0..."


In [34]:
homologables.shape

(212, 2)

### preparación para visualización de la homologación de capitulos

In [36]:
edt_base = pd.read_csv("base_insumos_2.csv")
edt_base.head()

,__hoja,Código,Descripción,Padre,Tipo,Descripción_prefijada,embedding
0,Hoja1,CD,COSTOS DIRECTOS,NaN,Capítulo,COSTOS DIRECTOS,"[0.0011236402206122875, 0.02140445075929165, 0..."
1,Hoja1,1,PRELIMINARES,NaN,Capítulo,PRELIMINARES,"[0.037456341087818146, 0.010043478570878506, 0..."
2,Hoja1,1.01,Obras Preliminares,1.00,Subcapítulo,PRELIMINARES. Obras Preliminares,"[0.03775278106331825, 0.019620878621935844, 0...."
3,Hoja1,1.01.001,Localización y replanteo,1.01,Ítem,PRELIMINARES. Obras Preliminares. Localización...,"[0.03098735399544239, 0.05125334486365318, 0.0..."
4,Hoja1,1.01.010,Aseo y limpieza lote,1.01,Ítem,PRELIMINARES. Obras Preliminares. Aseo y limpi...,"[0.03970905765891075, 0.012599264271557331, 0...."


In [37]:
# --- join homologation mapping to tabla_join ---
edt_base["Capitulo"] = (
    edt_base["Código"].astype(str).str.strip()
    + ". "
    + edt_base["Descripción"].astype(str).str.strip()
)

In [63]:
embedding_capitulos_base = edt_base[edt_base["Tipo"] == "Capítulo"]
embedding_capitulos_base = embedding_capitulos_base[embedding_capitulos_base["Descripción"] != "COSTOS DIRECTOS"]
embedding_capitulos_base = embedding_capitulos_base[["Capitulo","embedding"]]

In [50]:
embedding_capitulos_base.head()

,Capitulo,embedding
1,1. PRELIMINARES,"[0.037456341087818146, 0.010043478570878506, 0..."
110,2. CIMENTACION,"[0.023605743423104286, 0.037307996302843094, 0..."
598,3. DESAGUES,"[0.04226625710725784, 0.019774680957198143, 0...."
618,4. ESTRUCTURA,"[0.014902272261679173, 0.021512864157557487, 0..."
1098,12. INSTALACIONES HIDROSANITARIAS,"[-0.016090018674731255, 0.01740829274058342, 0..."


#### concatenación

In [64]:
# --- standardize column names ---
embedding_capitulos_base = embedding_capitulos_base.rename(columns={"Capitulo": "text", "embedding": "vector"})
homologables = homologables.rename(columns={"text": "text", "vector": "vector"})

# --- add Tipo column ---
embedding_capitulos_base["Tipo"] = "base"
homologables["Tipo"] = "homologable"

# --- concatenate ---
embedding_capitulos = pd.concat([embedding_capitulos_base, homologables], ignore_index=True)
print("[DEBUG] Combined shape:", embedding_capitulos.shape)
embedding_capitulos.head(1000)


[DEBUG] Combined shape: (249, 3)


,text,vector,Tipo
0,1. PRELIMINARES,"[0.037456341087818146, 0.010043478570878506, 0...",base
1,2. CIMENTACION,"[0.023605743423104286, 0.037307996302843094, 0...",base
2,3. DESAGUES,"[0.04226625710725784, 0.019774680957198143, 0....",base
3,4. ESTRUCTURA,"[0.014902272261679173, 0.021512864157557487, 0...",base
4,12. INSTALACIONES HIDROSANITARIAS,"[-0.016090018674731255, 0.01740829274058342, 0...",base
...,...,...,...
244,22 00 00. INSTALACIONES HIDROSANITARIAS,"[-0.029371976852416992, 0.021263960748910904, ...",homologable
245,04 00 00. MAMPOSTERIA,"[0.013060507364571095, 0.01295626163482666, -0...",homologable
246,02 00 00. CONDICIONES EXISTENTES,"[-0.01917501725256443, 0.035235896706581116, 0...",homologable
247,31 00 00. EXCAVACION Y CIMENTACION,"[-0.006594824604690075, 0.04966232180595398, 0...",homologable


### homologación de capitulos

In [ ]:
# --- build homologation queries from tabla_join ---
pares_lista = (
    tabla_join[["Capitulo Numero", "Capitulo Descripcion"]]
    .drop_duplicates()
    .apply(lambda fila: f"{fila['Capitulo Numero']}. {fila['Capitulo Descripcion']}", axis=1)
    .tolist()
)

# --- create homologation mapping dataframe ---
mapping = pd.read_csv("propuesta_homologacion_capitulos.csv")[
    ["query", "codigo_candidato", "descripcion_candidato"]
]

# --- join homologation mapping to tabla_join ---
tabla_join["query"] = (
    tabla_join["Capitulo Numero"].astype(str).str.strip()
    + ". "
    + tabla_join["Capitulo Descripcion"].astype(str).str.strip()
)

tabla_join = tabla_join.merge(mapping, on="query", how="left")

# --- assign homologated columns ---
tabla_join["Capitulo Numero Homologado"] = tabla_join["codigo_candidato"]
tabla_join["Capitulo Descripcion Homologada"] = tabla_join["descripcion_candidato"]

print("[DEBUG] ✅ Homologation complete.")
print("[DEBUG] Unmatched rows:", tabla_join["Capitulo Numero Homologado"].isna().sum())


In [ ]:
tabla_join.drop(columns=["query", "codigo_candidato", "descripcion_candidato"], inplace=True)


In [12]:
tabla_join.head(100)

,SkIdEmpresa_x,SkIdProyecto,SkIdCapitulo,SkIdItems,SkIdInsumo,SkIdReforma,SkIdUsuario,SkIdFecha,SkIdFecha Real,SkIdEstado,...,Insumo_Fecha Creacion,Insumo_Fecha Modificacion,Insumo_Codigo Insumo Id,query,codigo_candidato_x,descripcion_candidato_x,Capitulo Numero Homologado,Capitulo Descripcion Homologada,codigo_candidato_y,descripcion_candidato_y
0,100,1005,1005100,1006583,100279,NaN,100103,20110929,20110929,100101,...,06/12/2010,13/01/2011,279.0,2. CIMENTACION,2,CIMENTACION,2,CIMENTACION,2,CIMENTACION
1,100,1005,1005100,1006583,1002087,NaN,100103,20110929,20110929,100101,...,06/12/2010,NaN,2087.0,2. CIMENTACION,2,CIMENTACION,2,CIMENTACION,2,CIMENTACION
2,100,1005,1005100,1006583,1006063,NaN,100103,20110929,20110929,100101,...,06/12/2010,27/05/2011,6063.0,2. CIMENTACION,2,CIMENTACION,2,CIMENTACION,2,CIMENTACION
3,100,1005,1005100,1007643,100279,NaN,100103,20110930,20110930,100101,...,06/12/2010,13/01/2011,279.0,2. CIMENTACION,2,CIMENTACION,2,CIMENTACION,2,CIMENTACION
4,100,1005,1005100,1007643,1002087,NaN,100103,20110930,20110930,100101,...,06/12/2010,NaN,2087.0,2. CIMENTACION,2,CIMENTACION,2,CIMENTACION,2,CIMENTACION
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,100,1005,1005102,1005464,1002329,NaN,100103,20111021,20111021,100101,...,06/12/2010,NaN,2329.0,4. ESTRUCTURA,4,ESTRUCTURA,4,ESTRUCTURA,4,ESTRUCTURA
96,100,1005,1005102,1005464,1002400,NaN,100103,20111021,20111021,100101,...,06/12/2010,NaN,2400.0,4. ESTRUCTURA,4,ESTRUCTURA,4,ESTRUCTURA,4,ESTRUCTURA
97,100,1005,1005102,1005464,1003845,NaN,100103,20111021,20111021,100101,...,06/12/2010,26/05/2011,3845.0,4. ESTRUCTURA,4,ESTRUCTURA,4,ESTRUCTURA,4,ESTRUCTURA
98,100,1005,1005102,1005464,1005022,NaN,100103,20111021,20111021,100101,...,06/12/2010,29/12/2022,5022.0,4. ESTRUCTURA,4,ESTRUCTURA,4,ESTRUCTURA,4,ESTRUCTURA


## reducción dimensional

### v2

In [65]:
import numpy as np, json, ast, re

print("[DEBUG] Normalizing vector column formats...")

def parse_vector(value):
    if isinstance(value, (list, tuple, np.ndarray)):
        return np.asarray(value, dtype=np.float32).tolist()
    s = str(value).strip()
    if not s or s.lower() == "nan":
        return np.nan
    # try JSON first: "[0.1, 0.2, ...]"
    try:
        return np.asarray(json.loads(s), dtype=np.float32).tolist()
    except Exception:
        pass
    # try Python literal: "(0.1, 0.2)" or "[0.1 0.2]" variants
    try:
        return np.asarray(ast.literal_eval(s), dtype=np.float32).tolist()
    except Exception:
        pass
    # fallback: strip brackets and split by comma OR whitespace
    s = s.strip("[]()")
    parts = re.split(r"[,\s]+", s)
    nums = [float(p) for p in parts if p]
    return np.asarray(nums, dtype=np.float32).tolist()

embedding_capitulos["vector"] = embedding_capitulos["vector"].apply(parse_vector)

# sanity check: consistent dimensionality
lengths = embedding_capitulos["vector"].apply(lambda v: len(v) if isinstance(v, list) else -1)
if (lengths <= 0).any():
    bad_rows = embedding_capitulos[lengths <= 0][["text","Tipo","vector"]]
    print(f"[DEBUG] Found {len(bad_rows)} unparsable vectors.")
mode_dim = int(lengths[lengths > 0].mode().iat[0])
bad_dim = embedding_capitulos[lengths != mode_dim]
print(f"[DEBUG] Target dim: {mode_dim} | Rows with wrong dim: {len(bad_dim)}")

# optional: drop inconsistent rows to proceed safely
embedding_capitulos = embedding_capitulos[lengths == mode_dim].copy()


[DEBUG] Normalizing vector column formats...
[DEBUG] Target dim: 1536 | Rows with wrong dim: 0


In [66]:
embedding_capitulos.head(100)

,text,vector,Tipo
0,1. PRELIMINARES,"[0.037456341087818146, 0.010043478570878506, 0...",base
1,2. CIMENTACION,"[0.023605743423104286, 0.037307996302843094, 0...",base
2,3. DESAGUES,"[0.04226625710725784, 0.019774680957198143, 0....",base
3,4. ESTRUCTURA,"[0.014902272261679173, 0.021512864157557487, 0...",base
4,12. INSTALACIONES HIDROSANITARIAS,"[-0.016090018674731255, 0.01740829274058342, 0...",base
...,...,...,...
95,32. EXTERIORES,"[0.0133900111541152, 0.02780785597860813, 0.03...",homologable
96,8. ABERTURAS Y FACHADAS,"[0.040190670639276505, 0.07305998355150223, 0....",homologable
97,60. IMPREVISTOS,"[0.02255389094352722, 0.020186442881822586, -0...",homologable
98,23. INSTALACIONES HVAC,"[-0.023532113060355186, 0.020288905128836632, ...",homologable


In [71]:
import pandas as pd
import numpy as np
import joblib
import umap
import hdbscan
from datetime import datetime
from pathlib import Path
# --- UMAP + HDBSCAN simple pipeline (Parquet version) ---
import numpy as np, umap, hdbscan, pandas as pd
from datetime import datetime

print("[DEBUG] Building array from embeddings...")
X = np.vstack(embedding_capitulos["vector"])

[DEBUG] Building array from embeddings...


In [72]:
print("[DEBUG] Fitting UMAP...")
u = umap.UMAP(n_neighbors=15, n_components=2, metric="cosine", random_state=42).fit(X)

print("[DEBUG] Fitting HDBSCAN...")
h = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=2).fit(u.embedding_)

print("[DEBUG] Appending results...")
embedding_capitulos["umap_x"] = u.embedding_[:, 0]
embedding_capitulos["umap_y"] = u.embedding_[:, 1]
embedding_capitulos["cluster"] = h.labels_
embedding_capitulos["strength"] = h.probabilities_
print("[DEBUG] Masking base rows from analysis...")
# make cluster_mod categorical (no numeric binning


[DEBUG] Fitting UMAP...


C:\Users\aleja\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\aleja\AppData\Roaming\Python\Python312\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


[DEBUG] Fitting HDBSCAN...
[DEBUG] Appending results...
[DEBUG] Masking base rows from analysis...


C:\Users\aleja\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\aleja\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [73]:
# make cluster_mod categorical (no numeric binning)
embedding_capitulos["cluster_mod"] = np.where(
    embedding_capitulos["Tipo"].eq("base"), "BASE",
    np.where(embedding_capitulos["cluster"].eq(-1), "NOISE",
             "C" + embedding_capitulos["cluster"].astype(str))
).astype(str)


out = f"exp_{datetime.now().strftime('%Y%m%dT%H%M%S')}.parquet"
embedding_capitulos.to_parquet(out, index=False)
print(f"✅ Saved {out}")


✅ Saved exp_20251110T203341.parquet


### v1

#### ejecución

In [ ]:
import pandas as pd
import numpy as np
import joblib
import umap
import hdbscan
from datetime import datetime
from pathlib import Path
# --- UMAP + HDBSCAN simple pipeline (Parquet version) ---
import numpy as np, umap, hdbscan, pandas as pd
from datetime import datetime

print("[DEBUG] Building array from embeddings...")
X = np.vstack(embedding_capitulos["vector"])

print("[DEBUG] Fitting UMAP...")
u = umap.UMAP(n_neighbors=15, n_components=2, metric="cosine", random_state=42).fit(X)

print("[DEBUG] Fitting HDBSCAN...")
h = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=2).fit(u.embedding_)

print("[DEBUG] Appending results...")
embedding_capitulos["umap_x"] = u.embedding_[:, 0]
embedding_capitulos["umap_y"] = u.embedding_[:, 1]
embedding_capitulos["cluster"] = h.labels_
embedding_capitulos["strength"] = h.probabilities_

out = f"exp_{datetime.now().strftime('%Y%m%dT%H%M%S')}.parquet"
embedding_capitulos.to_parquet(out, index=False)
print(f"✅ Saved {out}")


[DEBUG] Building array from embeddings...
[DEBUG] Fitting UMAP...


C:\Users\aleja\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\aleja\AppData\Roaming\Python\Python312\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


[DEBUG] Fitting HDBSCAN...
[DEBUG] Appending results...
✅ Saved exp_20251107T200903.parquet


C:\Users\aleja\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\aleja\AppData\Roaming\Python\Python312\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


#### exportación

In [26]:
import joblib

joblib.dump(df, "embeddings.joblib")

['embeddings.joblib']

In [8]:
import pandas as pd

# Replace with your generated filename
infile = "exp_20251107T200903.parquet"

df = pd.read_parquet(infile)
outfile = infile.replace(".parquet", ".csv")
df.to_csv(outfile, index=False)

print(f"✅ Converted {infile} → {outfile}")

✅ Converted exp_20251107T200903.parquet → exp_20251107T200903.csv


# CONCATENAR DESCRIPCIONES JERÁRQUICAS Y MOSTRAR MUESTRA

#### Concatenar

In [11]:
# === CONCATENAR DESCRIPCIONES JERÁRQUICAS Y MOSTRAR MUESTRA ===

import pandas as pd

def construir_descripcion_jerarquica(fila: pd.Series,
                                     columnas_a_concatenar: list[str],
                                     separador: str = " - ") -> str:
    partes_limpias: list[str] = []
    for nombre_columna in columnas_a_concatenar:
        valor = fila.get(nombre_columna, "")
        if pd.isna(valor):
            continue
        valor_str = str(valor).strip()
        if valor_str:
            partes_limpias.append(valor_str)
    return separador.join(partes_limpias)

columnas_objetivo: list[str] = [
    "Macroproyecto",
    "Proyecto_sin_prefijo",
    "Capitulo Descripcion",
    "Item Descripcion",
    "Insumo_Insumo Descripcion",
]

columnas_que_existen = [c for c in columnas_objetivo if c in tabla_join.columns]
columnas_que_faltan = [c for c in columnas_objetivo if c not in tabla_join.columns]

print(f"[DEBUG] Columnas objetivo: {columnas_objetivo}")
print(f"[DEBUG] Columnas encontradas en 'tabla_join': {columnas_que_existen}")
if columnas_que_faltan:
    print(f"[DEBUG] ADVERTENCIA: Faltan estas columnas y se omitirán: {columnas_que_faltan}")

nombre_columna_salida = "Descripcion_Jerarquica"
vista_concatenacion[nombre_columna_salida] = vista_concatenacion.apply(
    construir_descripcion_jerarquica,
    axis=1,
    columnas_a_concatenar=columnas_que_existen,
    separador=" - "
)





[DEBUG] Columnas objetivo: ['Macroproyecto', 'Proyecto_sin_prefijo', 'Capitulo Descripcion', 'Item Descripcion', 'Insumo_Insumo Descripcion']
[DEBUG] Columnas encontradas en 'tabla_join': ['Macroproyecto', 'Proyecto_sin_prefijo', 'Capitulo Descripcion', 'Item Descripcion', 'Insumo_Insumo Descripcion']


C:\Users\aleja\AppData\Local\Temp\ipykernel_25924\3746829230.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vista_concatenacion[nombre_columna_salida] = vista_concatenacion.apply(


In [12]:
# Vista rápida de verificación
#display(tabla_final[["Nombre Proyecto", "Macroproyecto"]].head(10))

# Obtener conjuntos únicos
set_macroproyectos = set(tabla_join["Proyecto_sin_prefijo"].dropna().str.strip())
# Filtrar filas donde el campo 'Macroproyecto' está vacío o no coincide con los esperados
filtro_no_macro = tabla_join[tabla_join["Macroproyecto"].notna() | (tabla_join["Macroproyecto"].str.strip() == "")]

# Mostrar solo algunas columnas relevantes
columnas_mostrar = ["Nombre Proyecto", "Macroproyecto", "Proyecto_sin_prefijo", "SkIdProyecto", "SkIdCapitulo"]
display(filtro_no_macro[columnas_mostrar].head(10))

,Nombre Proyecto,Macroproyecto,Proyecto_sin_prefijo,SkIdProyecto,SkIdCapitulo
10273,Caminos de Sie - Urb y Zonas Comunes MZ 4,Caminos de Sie - Manzana 4,Caminos de Sie - Urb y Zonas Comunes MZ 4,10028,10028576
10274,Caminos de Sie - Urb y Zonas Comunes MZ 4,Caminos de Sie - Manzana 4,Caminos de Sie - Urb y Zonas Comunes MZ 4,10028,10028576
10275,Caminos de Sie - Urb y Zonas Comunes MZ 4,Caminos de Sie - Manzana 4,Caminos de Sie - Urb y Zonas Comunes MZ 4,10028,10028576
10276,Caminos de Sie - Urb y Zonas Comunes MZ 4,Caminos de Sie - Manzana 4,Caminos de Sie - Urb y Zonas Comunes MZ 4,10028,10028576
10277,Caminos de Sie - Urb y Zonas Comunes MZ 4,Caminos de Sie - Manzana 4,Caminos de Sie - Urb y Zonas Comunes MZ 4,10028,10028576
12252,Caminos de Sie - Urb y Zonas Comunes MZ 4,Caminos de Sie - Manzana 4,Caminos de Sie - Urb y Zonas Comunes MZ 4,10028,10028603
12358,Caminos de SIE - Edificaciones MZ 4,Caminos de Sie - Manzana 4,Caminos de SIE - Edificaciones MZ 4,10013,10013289
12359,Caminos de SIE - Edificaciones MZ 4,Caminos de Sie - Manzana 4,Caminos de SIE - Edificaciones MZ 4,10013,10013289
12360,Caminos de SIE - Edificaciones MZ 4,Caminos de Sie - Manzana 4,Caminos de SIE - Edificaciones MZ 4,10013,10013289
12361,Caminos de SIE - Edificaciones MZ 4,Caminos de Sie - Manzana 4,Caminos de SIE - Edificaciones MZ 4,10013,10013289


In [13]:
# Mostrar solo filas con Macroproyecto definido
tamano_muestra: int = 100
muestra_pequena = (
    vista_concatenacion[
        vista_concatenacion["Macroproyecto"].notna()
        & (vista_concatenacion["Macroproyecto"].astype(str).str.strip() != "")
    ][[nombre_columna_salida]]
    .head(tamano_muestra)
)

print(f"[DEBUG] Tamaño de la tabla con la nueva columna: {vista_concatenacion.shape}")
print(f"[DEBUG] Mostrando las primeras {tamano_muestra} filas que tienen Macroproyecto definido:")
display(muestra_pequena)

[DEBUG] Tamaño de la tabla con la nueva columna: (273450, 5)
[DEBUG] Mostrando las primeras 100 filas que tienen Macroproyecto definido:


,Descripcion_Jerarquica
10273,Caminos de Sie - Manzana 4 - Caminos de Sie - ...
10274,Caminos de Sie - Manzana 4 - Caminos de Sie - ...
10275,Caminos de Sie - Manzana 4 - Caminos de Sie - ...
10276,Caminos de Sie - Manzana 4 - Caminos de Sie - ...
10277,Caminos de Sie - Manzana 4 - Caminos de Sie - ...
...,...
12477,Caminos de Sie - Manzana 4 - Caminos de SIE - ...
12478,Caminos de Sie - Manzana 4 - Caminos de SIE - ...
12479,Caminos de Sie - Manzana 4 - Caminos de SIE - ...
12480,Caminos de Sie - Manzana 4 - Caminos de SIE - ...


#### debug

In [28]:
# === DIAGNÓSTICO RÁPIDO DE MACROPROYECTOS VACÍOS ===

print("[DEBUG] Total de filas:", len(tabla_looker))
print("[DEBUG] Filas con Macroproyecto no nulo:",
      tabla_looker["Macroproyecto"].notna().sum())

# ¿Qué valores distintos hay en Macroproyecto?
print("[DEBUG] Valores únicos de Macroproyecto (primeros 10):")
print(tabla_looker["Macroproyecto"].dropna().unique()[:10])

# Si todos son NaN o '', veamos qué pasa con los nombres:
proyectos_en_tabla = set(tabla_looker["Nombre Proyecto"].dropna().unique())
ruta_macro = r"20251003/20251003/proyectos_macroproyectos.csv"
tabla_macro = pd.read_csv(ruta_macro)

# Detectar nombres coincidentes, sin normalizar
col_macro_nombre = "Proyecto" if "Proyecto" in tabla_macro.columns else "Proyectos"
proyectos_en_macro = set(tabla_macro[col_macro_nombre].dropna().unique())

interseccion = proyectos_en_tabla.intersection(proyectos_en_macro)
print(f"[DEBUG] Coincidencias exactas entre nombres: {len(interseccion)}")

if len(interseccion) == 0:
    print("[DEBUG] No hay coincidencias exactas de texto — probablemente difieren en mayúsculas, acentos o espacios.")


[DEBUG] Total de filas: 273450
[DEBUG] Filas con Macroproyecto no nulo: 0
[DEBUG] Valores únicos de Macroproyecto (primeros 10):
[]
[DEBUG] Coincidencias exactas entre nombres: 0
[DEBUG] No hay coincidencias exactas de texto — probablemente difieren en mayúsculas, acentos o espacios.


#### exportar

In [26]:
# === GUARDAR TABLA EN PARQUET ===
import os

# Ruta y nombre de salida (usa mismo nombre base que el CSV)
nombre_archivo_parquet = "tabla_descripcion_jerarquica.parquet"
ruta_salida_parquet = os.path.join(os.getcwd(), nombre_archivo_parquet)

# Guardar en formato Parquet
try:
    vista_concatenacion.to_parquet(ruta_salida_parquet, index=False)
    print(f"[DEBUG] Archivo guardado correctamente en formato Parquet: {ruta_salida_parquet}")
    print(f"[DEBUG] Tamaño de la tabla guardada: {vista_concatenacion.shape}")
except Exception as error:
    print(f"[DEBUG] ERROR al guardar el archivo Parquet: {error}")


try:
    vista_concatenacion.to_csv("tabla_descripcion_jerarquica.csv", index=False, encoding="utf-8", sep=",")
    print(f"[DEBUG] Archivo guardado correctamente en formato CSV: tabla_descripcion_jerarquica.csv")
    print(f"[DEBUG] Tamaño de la tabla guardada: {vista_concatenacion.shape}")
except Exception as error:
    print(f"[DEBUG] ERROR al guardar el archivo CSV: {error}")


[DEBUG] Archivo guardado correctamente en formato Parquet: c:\Users\aleja\Documents\Ingenieria Estadistica\Asignaturas2025B\arpro1\Base de Datos ARPRO\tabla_descripcion_jerarquica.parquet
[DEBUG] Tamaño de la tabla guardada: (273450, 5)
[DEBUG] Archivo guardado correctamente en formato CSV: tabla_descripcion_jerarquica.csv
[DEBUG] Tamaño de la tabla guardada: (273450, 5)


# Concatenar hasta items

In [5]:
# === CONCATENAR DESCRIPCIONES JERÁRQUICAS Y MOSTRAR MUESTRA ===

import pandas as pd

def construir_descripcion_jerarquica(fila: pd.Series,
                                     columnas_a_concatenar: list[str],
                                     separador: str = " - ") -> str:
    partes_limpias: list[str] = []
    for nombre_columna in columnas_a_concatenar:
        valor = fila.get(nombre_columna, "")
        if pd.isna(valor):
            continue
        valor_str = str(valor).strip()
        if valor_str:
            partes_limpias.append(valor_str)
    return separador.join(partes_limpias)

columnas_objetivo: list[str] = [
    "Macroproyecto",
    "Proyecto_sin_prefijo",
    "Capitulo Descripcion",
    "Item Descripcion",
    
]

columnas_que_existen = [c for c in columnas_objetivo if c in tabla_join.columns]
columnas_que_faltan = [c for c in columnas_objetivo if c not in tabla_join.columns]

print(f"[DEBUG] Columnas objetivo: {columnas_objetivo}")
print(f"[DEBUG] Columnas encontradas en 'tabla_join': {columnas_que_existen}")
if columnas_que_faltan:
    print(f"[DEBUG] ADVERTENCIA: Faltan estas columnas y se omitirán: {columnas_que_faltan}")

nombre_columna_salida = "Descripcion_Jerarquica"
vista_concatenacion[nombre_columna_salida] = vista_concatenacion.apply(
    construir_descripcion_jerarquica,
    axis=1,
    columnas_a_concatenar=columnas_que_existen,
    separador=" - "
)





[DEBUG] Columnas objetivo: ['Macroproyecto', 'Proyecto_sin_prefijo', 'Capitulo Descripcion', 'Item Descripcion']
[DEBUG] Columnas encontradas en 'tabla_join': ['Macroproyecto', 'Proyecto_sin_prefijo', 'Capitulo Descripcion', 'Item Descripcion']


C:\Users\aleja\AppData\Local\Temp\ipykernel_28176\2772702638.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vista_concatenacion[nombre_columna_salida] = vista_concatenacion.apply(


In [8]:
# Mostrar cantidad de valores únicos de la nueva columna
valores_unicos = vista_concatenacion[nombre_columna_salida].nunique()
print(f"[DEBUG] Cantidad de valores únicos en '{nombre_columna_salida}': {valores_unicos}")


[DEBUG] Cantidad de valores únicos en 'Descripcion_Jerarquica': 25225
